# DEMO: AskYourBooks Prototyp with Gradio and Llama LLM

## Installation der nötigen Bibliotheken

In [1]:
!python --version

Python 3.10.12


In [2]:
!pip -q install gradio
!pip -q install langchain
!pip -q install llama-cpp-python
!pip -q install transformers
!pip -q install sentence_transformers
!pip -q install tiktoken
!pip -q install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.9 MB/s eta 0

In [ ]:
!git lfs install --skip-repo

Git LFS initialized.


## Import der nötigen Python Module

In [3]:
import os
import requests
import io
import random
import time
from getpass import getpass
from langchain import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain import PromptTemplate, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

## Wichtige Konstanten

In [4]:
# Enter your Hugging Face API Token here
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass()
REPO_ID = "declare-lab/flan-alpaca-large"
#REPO_ID = "decapoda-research/llama-7b-hf"



··········


In [10]:
#TEXT_URL = "https://www.gutenberg.org/cache/epub/164/pg164.txt"
TEXT_URL ="https://www.gutenberg.org/ebooks/164.txt.utf-8"

## Sprachmodell und Embeddings initialisieren

In [5]:
llm = HuggingFaceHub(repo_id=REPO_ID, model_kwargs={"temperature":0, "max_length":512})

In [6]:
embeddings = HuggingFaceEmbeddings()

In [7]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [8]:
chain = load_qa_chain(llm, chain_type="stuff", verbose=False)

## Text einlesen und verarbeiten

In [11]:
# Get text
res = requests.get(TEXT_URL)
with open("pg164.txt", "w") as f:
  f.write(res.text)

In [12]:
# Load and split text
loader = TextLoader("./pg164.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

## Embedings erzeugen und in Vektordatenbank speichern

In [13]:
db = FAISS.from_documents(docs, embeddings)

In [14]:
db.save_local(folder_path=".", index_name="faiss_index")

## Fragen an den Text

In [18]:
question = "What is the name of the captain of the Nautilus"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'Captain Nemo.'

In [16]:
question = "What are the names of the crew members of the Nautilus?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'Captain Nemo, Conseil, and the Canadian.'

In [17]:
question = "What is the name of the Canadian?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'Ned Land.'

In [19]:
question = "What musical instrument does Captain Nemo play?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'Captain Nemo plays the organ.'

In [ ]:
question = "What is the name of Professor Aronnax's butler?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'Conseil.'

In [ ]:
question = "In which areas was the Nautilus traveling?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

'The Nautilus was traveling in the Indian Ocean.'

In [ ]:
question = "Why doesn't Captain Nemo like people?"
relevant_docs = db.similarity_search(question)
chain.run(input_documents=relevant_docs, question=question, raw_response=True, callbacks=callback_manager)

"No, Captain Nemo doesn't like people because he is not satisfied with shunning man."

## Gradio Anwendung

In [20]:
import gradio as gr

In [21]:
question_examples = [
    "What is the name of the captain of the Nautilus?",
    "What are the names of the crew members of the Nautilus?",
    "What is the name of the Canadian?",
    "What is Ned Land's profession?",
    "What musical instrument does Captain Nemo play?",
    "What is the name of Professor Aronnax's butler?",
    "In which areas was the Nautilus traveling?",
    "Why doesn't Captain Nemo hate the society?"
]

In [22]:
def read_textfile(File):
  loader = TextLoader(File.name)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
  docs = text_splitter.split_documents(documents)
  db = FAISS.from_documents(docs, embeddings)
  db.save_local(folder_path=".", index_name="faiss_index")

In [23]:
def ask_question(question, chat_history):
  llm = HuggingFaceHub(repo_id=REPO_ID, model_kwargs={"temperature":0, "max_length":512})
  embeddings = HuggingFaceEmbeddings()
  chain = load_qa_chain(llm, chain_type="stuff", verbose=False)
  db = FAISS.load_local(".", embeddings, index_name="faiss_index")
  relevant_docs = db.similarity_search(question)

  answer = chain.run(
      input_documents=relevant_docs,
      question=question,
      callbacks=callback_manager
  )

  chat_history.append((question, answer))
  time.sleep(1)
  return "", chat_history

In [24]:
with gr.Blocks() as demo:
  gr.Markdown("## 📚AskYourBooks")
  gr.Markdown("""
    Ask questions to your books in txt format. After a short indexing time,
    ask questions about the content of the book.
    """
  )
  textfile = gr.File(label="Drag your book here")
  ask_textbox = gr.Textbox(label="Question")
  sample_questions = gr.Examples(examples=question_examples, inputs=[ask_textbox])
  ask_button = gr.Button("Ask your book")
  clear_button = gr.Button("Clear chat")
  chatbot = gr.Chatbot(label="Conversation")

  textfile.change(fn=read_textfile, inputs=[textfile], outputs=[])
  ask_button.click(
      fn=ask_question,
      inputs=[ask_textbox, chatbot],
      outputs=[ask_textbox, chatbot]
  )
  clear_button.click(lambda: None, None, chatbot, queue=False)


In [ ]:
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>